In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df_continents = pd.read_csv(
    os.path.join("Data", "continents.csv"), index_col=0
)

display(df_continents)

### Colors for all plots

In [ ]:
continents = df_continents['continent'].unique()
random_colors = sns.color_palette('husl', n_colors=len(continents))
continents_colors = {}
for i in range(len(continents)):
    continents_colors[continents[i]] = random_colors[i]
print(continents_colors)

### Articles per continent

In [ ]:
continents_grouped = df_continents.groupby(['continent']).size()
continents_grouped = continents_grouped.sort_values(ascending=False)
continents_grouped

In [ ]:
plots_path = "plots"

fig = px.bar(
    x=list(continents_grouped.keys()),
    y=list(continents_grouped.values),
    labels={"x": "Continent", "y": "Count"},
    title="Count of Categories by Continent",
    color=list(continents_grouped.keys()),
)

# fig.update_layout(xaxis=dict(tickangle=-45))
fig.update_layout(showlegend=False)

file_path = os.path.join(plots_path, "count_of_categories_by_continent.html")
fig.write_html(file_path)

fig.show()


## Number of articles by category

In [ ]:
categories = pd.read_csv(
    os.path.join("Data", "wikispeedia_paths-and-graph", "categories.tsv"),
    delimiter="\t",
    header=None,
    names=["article", "category"],
    skip_blank_lines=True,
    comment="#",
    encoding="UTF-8"
)

display(categories.head())
print("Size:",categories.shape)

In [ ]:
articles = pd.merge(categories, df_continents, on="article", how="inner")
articles = articles.fillna("")
display(articles.head())
print("Size:",articles.shape)

In [ ]:
main_cats = []
for category in articles['category'].values:
    main_cats.append(category.split('.')[1])

articles['main_category'] = main_cats
display(articles)
print("Size:",articles.shape)

In [ ]:
articles_cat_continent = articles.groupby(['main_category','continent']).size()
display(articles_cat_continent[('Geography','International')])

I have to check this plot

In [ ]:
articles_cat_continent

In [ ]:
categories = articles['main_category'].unique()
continents = articles['continent'].unique()
continents.sort()

#In case we do not want international
mask = (continents != 'International')
continents = continents[mask]

category_positions = np.arange(len(categories))
bar_width = 0.5

continents_values = {}
for i, continent in enumerate(continents):
    frequencies = []
    for category in categories:
        try:
            value = articles_cat_continent[(category, continent)]
        except KeyError:
            value = 0
        frequencies.append(value)
    continents_values[continent] = frequencies

df_continent_frequencies = pd.DataFrame(continents_values).T
df_continent_frequencies.columns = categories
display(df_continent_frequencies)
print("Size:", df_continent_frequencies.shape)

fig = px.bar(
    df_continent_frequencies.T,
    orientation='h',
    title='Frequency of Continents in Each Category',
    labels={"index": "Category", "value": "Frequency"},
    #category_orders={"index": categories},
)

file_path = os.path.join(plots_path, "frequency_continents_in_each_category.html")
fig.write_html(file_path)

fig.show()



## Length of articles

In [ ]:
articles_df = pd.read_csv("Data/wikispeedia_paths-and-graph/articles.tsv", sep="\t",header=None, names=["article"], skiprows=11)
display(articles_df.head())
print("Size:",articles_df.shape)

In [ ]:
# Compute the length and create a new dataframe called analysis_df to be used during the initial analysis

analysis_df = articles_df.copy()

# Specify the path to the folder containing the .txt files
plaintext_path = 'data/plaintext_articles'

# Create an empty list to store the results
word_counts = []

# Iterate through each article in the articles.tsv file
for index, row in analysis_df.iterrows():
    # Construct the full path to the .txt file
    file_path = os.path.join(plaintext_path, row['article'] + '.txt')

    # Read the contents of the .txt file
    with open(file_path, 'r', encoding='utf-8') as file:

        _ = file.readline() # Skip the first line because it contains the word #copyright
        content = file.read()

    # Count the number of words in the article
    word_count = len(content.split())

    # Append the result to the list
    word_counts.append(word_count)

# Add a new column 'WordCount' to the analysis_df
analysis_df['length'] = word_counts

In [ ]:
display(analysis_df.head())
print("Size:",analysis_df.shape)

In [ ]:
# Merge analysis with continents
analysis_df = pd.merge(analysis_df, df_continents, on=["article"])
display(analysis_df.head())
print("Size:",analysis_df.shape)

In [ ]:
#Compute the average length of article for each continent
continent_length = analysis_df.groupby(['continent']).mean('length')
continent_length=continent_length.sort_values('length',ascending=False)
display(continent_length)
print("Size:",continent_length.shape)

In [ ]:
fig = px.bar(
    x=continent_length.index,
    y=continent_length.values.reshape(8),
    color=list(continents_grouped.keys()),
    labels={"x": "Continent", "y": "Length"},
    title="Average length of articles by continent",
)

fig.update_layout(showlegend=False)

file_path = os.path.join(plots_path, "average_length_of_articles_by_continent.html")
fig.write_html(file_path)

fig.show()

## Pagerank

In [ ]:
# Load pagerank
pagerank_df = pd.read_csv("Data/page_rank.csv")

In [ ]:
display(pagerank_df.head())
print("Size:", pagerank_df.shape)
print("Note that the pageRank has less rows!!")


In [ ]:
analysis_df = pd.merge(analysis_df, pagerank_df, left_on=["article"], right_on=["Article"], how="left").fillna(0)
analysis_df = analysis_df.drop(["Article"], axis=1)

In [ ]:
display(analysis_df.head())
print("Size:", analysis_df.shape)

In [ ]:
# Compute mean
mean_pagerank_continent = analysis_df.groupby("continent").mean("PageRank")
mean_pagerank_continent =mean_pagerank_continent.sort_values(by='PageRank', ascending = False)

In [ ]:
fig = px.bar(
    x=mean_pagerank_continent.index,
    y=mean_pagerank_continent['PageRank'],
    color=list(continents_grouped.keys()),
    labels={"x": "Continent", "y": "Pagerank"},
    title="Average PageRank by continent",
)

fig.update_layout(showlegend=False)
fig.update_yaxes(tickformat=".1e")

file_path = os.path.join(plots_path, "average_pagerank_by_continent.html")
fig.write_html(file_path)

fig.show()

## Analysis of paths

Group paths by continents, one path will be assigned to the continent corresponding to the GOAL article

Compute the number of "backclicks" in each path

Compute the length of each path

In [ ]:
paths_finished_df = pd.read_csv("Data/wikispeedia_paths-and-graph/paths_finished.tsv", sep="\t", header=None, names = ["hashedIpAddress", "timestamp", "durationInSec", "path", "rating"], skiprows=16)
paths_unfinished_df = pd.read_csv("Data/wikispeedia_paths-and-graph/paths_unfinished.tsv", sep="\t", header=None, names = ["hashedIpAddress","timestamp","durationInSec", "unf_path", "target","motif"],skiprows=17)

# Extract the target article from finished paths
paths_finished_df["target"] = paths_finished_df['path'].apply(lambda x: x.split(';')[-1])

# Count the number of backclicks
paths_finished_df['backclicks'] = paths_finished_df['path'].apply(lambda x: x.count('<'))
paths_unfinished_df['backclicks'] = paths_unfinished_df['unf_path'].apply(lambda x: x.count('<'))

# Compute the length of each path
paths_finished_df['path_steps'] = paths_finished_df['path'].apply(lambda x: x.count(';') + 1 + x.count('<'))
paths_unfinished_df['path_steps'] = paths_unfinished_df['unf_path'].apply(lambda x: x.count(';') + 1 + x.count('<'))

In [ ]:
# Merge the paths with the continents

analysis_fin_paths = pd.merge(paths_finished_df, df_continents, left_on="target", right_on="article").drop("article", axis=1)
analysis_unf_paths = pd.merge(paths_unfinished_df, df_continents, left_on="target", right_on="article").drop("article", axis=1)

In [ ]:
display(analysis_fin_paths.head())
print("Size:", analysis_fin_paths.shape)

display(analysis_unf_paths.head())
print("Size:", analysis_unf_paths.shape)

In [ ]:
continent_data = pd.DataFrame()

# Create a column to show the number of articles in each continent
continent_data["n_articles"] = analysis_df.groupby("continent")["article"].count()

# Create column with the average PageRank
continent_data["av_pagerank"] = mean_pagerank_continent['PageRank']

# Create columns for av. backclicks
continent_data["av_fin_backcliks"] =  analysis_fin_paths.groupby("continent")["backclicks"].mean()
continent_data["av_unf_backcliks"] =  analysis_unf_paths.groupby("continent")["backclicks"].mean()

# Create columns for av. path steps
continent_data["av_fin_steps"] =  analysis_fin_paths.groupby("continent")["path_steps"].mean()
continent_data["av_unf_steps"] =  analysis_unf_paths.groupby("continent")["path_steps"].mean()

# Create columns for the number of finished and unfinished paths for each continent
continent_data["fin_paths"] = analysis_fin_paths["continent"].value_counts()
continent_data["unf_paths"] = analysis_unf_paths["continent"].value_counts()
continent_data['per_fin_paths'] = (continent_data["fin_paths"]/(continent_data["fin_paths"]+continent_data["unf_paths"]))*100 # I think not multiplying by 100

In [ ]:
continent_data=continent_data.sort_values(by='per_fin_paths',ascending=False)
display(continent_data)
print("Size:", continent_data.shape)

In [ ]:
fig = px.bar(
    x=continent_data.index,
    y=continent_data['per_fin_paths'],
    color=list(continents_grouped.keys()),
    labels={"x": "Continent", "y": "Percentage"},
    title="Percentage of finished paths by continent",
)

fig.update_layout(showlegend=False)

file_path = os.path.join(plots_path, "percentage_finished_paths_by_continent.html")
fig.write_html(file_path)

fig.show()

In [ ]:
total_paths = len(paths_finished_df) + len(paths_unfinished_df)

In [ ]:
# Create columns to count the number of times each article appears as goal in finished and unfinished paths
analysis_df["n_as_goal_in_fin"] = analysis_df["article"].map(analysis_fin_paths["target"].value_counts()).fillna(0)
analysis_df["n_as_goal_in_unf"] = analysis_df["article"].map(analysis_unf_paths["target"].value_counts()).fillna(0)

# Create columns to count the number of times each article appears in general in finished and unfinished paths
analysis_df["n_in_fin_paths"] = analysis_df["article"].map(pd.Series(';'.join(analysis_fin_paths['path']).split(';')).value_counts()).fillna(0)
analysis_df["n_in_unf_paths"] = analysis_df["article"].map(pd.Series(';'.join(analysis_unf_paths['unf_path']).split(';')).value_counts()).fillna(0)

#Let's calculate the probability of finding an article (we can compare this with the pagerank)
analysis_df["prob_finding"] = (analysis_df["n_in_fin_paths"] + analysis_df["n_in_unf_paths"])/total_paths

In [ ]:
display(analysis_df)
print("Size:", analysis_df.shape)

In [ ]:
# To do: Some plots
# Show the frequency of the articles